In [12]:
import plotly
plotly.__version__
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

import time

import dash
import dash_bootstrap_components as dbc
import numpy as np
import plotly.graph_objs as go
from dash import Input, Output, dcc, html

import plotly.express as px
import pandas as pd
import pathlib
from urllib.request import urlopen
import json

In [16]:
df1 = pd.read_csv("data\\bee_colony_survey_data_by_state.csv")
df1=df1[['year','period','week_ending','state','state_ansi', 'watershed', 'data_item', 'value']]
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))
#for state in df.state:
count = 0
for i in df1.state:
    for j in abbrev_to_us_state:
        if abbrev_to_us_state[j].lower()==i.lower():
            df1.loc[count,"state"] = j
    count=count+1

df1.rename({'value':'Beez'},axis=1, inplace=True)
df1=df1.sort_values("year") # Make sure you sort the time horizon column in ascending order because this column is in random order in the raw dataset


fig1 = px.choropleth(df1,
                    locations='state', 
                    locationmode="USA-states", 
                    color='Beez',
                    color_continuous_scale="Viridis_r", 
                    scope="usa",
                    range_color=(0,169000),
                    animation_frame='year') #make sure 'period_begin' is string type and sorted in ascending order


In [18]:
fig = make_subplots(rows=1, cols=2, subplot_titles = ('Subplot (1,1)', 'Subplot(1,2)'))

fig.add_trace(go.Scatter(
          x= [-2.0, -1.0, 0.01, 1.0, 2.0, 3.0],
          y= [4, 1, 1, 1, 4, 9],
          mode = 'markers+lines',
          hoverinfo='name',
          legendgroup= 'f1',
          line_color= 'rgb(255, 79, 38)',
          name= 'f1',
          showlegend= True), row=1, col=1);

fig.add_trace(go.Scatter(
          x= [-2.0, -1.0, 0.01, 1.0, 2.0, 3.0],
          y= [3, 8.3, 5.5, 4.24, 6.7, 7],
          mode = 'markers+lines',
          hoverinfo='name',
          legendgroup= 'f1p',
          line_color= 'rgb(79, 38, 255)',
          name= 'f1p',
          showlegend= True), row=1, col=1);

fig.add_trace(go.Scatter(
            x = [-2.0, -1.0, 0.01, 1.0, 2.0, 3.0],
            y = [2.5, 3, 6, 1, 2.5, 8.31],
            name= 'f2',
            showlegend= True,
            hoverinfo= 'name',
            legendgroup= 'f2',
            mode = 'markers+lines',
            line_color='green',
            ), row=1, col=2);

fig.layout

fig.update_layout(width=700, height=475)
fig.update_yaxes(range=[0, 9]);#this line updates both yaxis, and yaxis2 range

number_frames = 10
frames = [dict(
               name = k,
               data = [go.Scatter(y= 4+5*np.random.rand(6)),#update the trace 1 in (1,1)
                       go.Scatter(y = 2+6*np.random.rand(6)), #update the second trace in (1,1)
                       go.Scatter(y= 1+7*np.random.rand(10))#update the trace in (1,2)
                       ],
               traces=[0, 1, 2] # the elements of the list [0,1,2] give info on the traces in fig.data
                                      # that are updated by the above three go.Scatter instances
              ) for k in range(number_frames)]


updatemenus = [dict(type='buttons',
                    buttons=[dict(label='Play',
                                  method='animate',
                                  args=[[f'{k}' for k in range(10)], 
                                         dict(frame=dict(duration=500, redraw=False), 
                                              transition=dict(duration=0),
                                              easing='linear',
                                              fromcurrent=True,
                                              mode='immediate'
                                                                 )])],
                    direction= 'left', 
                    pad=dict(r= 10, t=85), 
                    showactive =True, x= 0.1, y= 0, xanchor= 'right', yanchor= 'top')
            ]

sliders = [{'yanchor': 'top',
            'xanchor': 'left', 
            'currentvalue': {'font': {'size': 16}, 'prefix': 'Frame: ', 'visible': True, 'xanchor': 'right'},
            'transition': {'duration': 500.0, 'easing': 'linear'},
            'pad': {'b': 10, 't': 50}, 
            'len': 0.9, 'x': 0.1, 'y': 0, 
            'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': False},
                                      'transition': {'duration': 0, 'easing': 'linear'}}], 
                       'label': k, 'method': 'animate'} for k in range(10)       
                    ]}]

fig.update(frames=frames),
fig.update_layout(updatemenus=updatemenus,
                  sliders=sliders);
fig.show() #in jupyter notebook
#fig.show('browser') # in browser (the former offline.plot)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'coloraxis': 'coloraxis',
              'geo': 'geo',
              'hovertemplate': 'year=1987<br>state=%{location}<br>Beez=%{z}<extra></extra>',
              'locationmode': 'USA-states',
              'locations': array(['WY', 'MI', 'MA', 'MD', 'ME', 'LA', 'KY', 'KS', 'IA', 'IN', 'IL', 'ID',
                                  'HI', 'GA', 'FL', 'DE', 'CT', 'CO', 'CA', 'AR', 'AZ', 'AL', 'MN', 'MS',
                                  'WI', 'MT', 'WV', 'WA', 'VA', 'VT', 'UT', 'TX', 'SD', 'SC', 'RI', 'PA',
                                  'TN', 'OK', 'OR', 'NV', 'NH', 'NJ', 'NM', 'NE', 'NC', 'ND', 'OH', 'NY',
                                  'MO'], dtype=object),
              'name': '',
              'type': 'choropleth',
              'z': array([ 39000,  93000,  11000,   7000,   9000,  35000,  14000,  46000,  44000,
                           25000,  280